# Investigating performance and scaling of CFDPINN and OpenFOAM

OpenFOAM is a highly optimized computational fluid dynamics software package.

In this notebook we compare and contrast OpenFOAM scalability against CFDPINN.

We begin by examining the simulation time using OpenFOAM for three examples of the lid-driven cavity case. These three cases use differing mesh sizes which in turn impacts the time to solution. The number of timesteps is held constant, as is the initial conditions.

- 20x20
- 100x100
- 200x200

We can then take the trained CFDPINN and perform inference for each of the three grid sizes.

- Load trained PINN model
- Setup features based on various geometries
- Perform inference with timing
- Compare accuracy

In [3]:
from cfdpinn.args import parse_args
import sys
import torch
from cfdpinn.geometry import setup_geom
from cfdpinn.pinns import predict_fluid
from cfdpinn.preprocess import merge_features
from pickle import load

In [4]:
pinn = torch.load("/Users/harry/pinns/models/tmp-2d-ns-pinn-1epochs-lr0.001-adam-cavity-nu0.01-u1-20x20.pt")

In [5]:
sys.argv = [
    "program-name",
    "--debug",
    "--case-type",
    "cavity",
    "--case-dir",
    "/Users/harry/pinns/openfoam/cases/cavity_nu0.01_U1",
    "--start-time",
    "0",
    "--end-time",
    "5",
    "--sim-dt",
    "0.001",
    "--load-dt",
    "0.005",
    "--numx",
    "20",
    "--numy",
    "20",
    "--viscosity",
    "0.01",
    "--initial_u",
    "1",
    "--scaler-path",
    "/Users/harry/pinns/models/tmp-scaler.pkl",
    "--model-path",
    "/Users/harry/pinns/models/tmp-2d-ns-pinn-1epochs-lr0.001-adam-cavity-nu0.01-u1-20x20.pt",
    "--epochs",
    "1",
    "--training-animation",
    "--prediction-animation",
    "--residual-animation",
    "--num-frames",
    "5",
    "--output-pred-data",
    "--output-train-data",
    "--output-data-path",
    "/Users/harry/pinns/data/tmp-outputs"
]

args_20x20 = parse_args()
geom_20x20 = setup_geom(args_20x20)

data = dict()
data["scaler"] = load(open("/Users/harry/pinns/models/tmp-scaler.pkl","rb"))

data = merge_features(data,geom_20x20)
data = predict_fluid(data,pinn,geom_20x20,args_20x20)

#################
##  CFD PINN   ##
#################

Setting up geometry...
Geometry summary:
	geom[x_start]: 0
	geom[x_end]: 1
	geom[numx]: 20
	geom[y_start]: 0
	geom[y_end]: 1
	geom[numy]: 20
	geom[t_start]: 0.0
	geom[t_end]: 5.0
	geom[t_dt]: 0.005
	geom[numt]: 1001
	geom[stride]: 5
	Geometry setup completed

Prediction of fluid properties...
	DEBUG: Applying scaler...
	DEBUG: Scaler applied
	DEBUG: len(x): 400400
	DEBUG: len(y): 400400
	DEBUG: len(t): 400400
	Fluid properties prediction complete



In [6]:
sys.argv = [
    "program-name",
    "--debug",
    "--case-type",
    "cavity",
    "--case-dir",
    "/Users/harry/pinns/openfoam/cases/cavity_nu0.01_U1",
    "--start-time",
    "0",
    "--end-time",
    "5",
    "--sim-dt",
    "0.001",
    "--load-dt",
    "0.001",
    "--numx",
    "100",
    "--numy",
    "100",
    "--viscosity",
    "0.01",
    "--initial_u",
    "1",
    "--scaler-path",
    "/Users/harry/pinns/models/tmp-scaler.pkl",
    "--model-path",
    "/Users/harry/pinns/models/tmp-2d-ns-pinn-1epochs-lr0.001-adam-cavity-nu0.01-u1-20x20.pt",
    "--epochs",
    "1",
    "--training-animation",
    "--prediction-animation",
    "--residual-animation",
    "--num-frames",
    "5",
    "--output-pred-data",
    "--output-train-data",
    "--output-data-path",
    "/Users/harry/pinns/data/tmp-outputs"
]

args_100x100 = parse_args()
geom_100x100 = setup_geom(args_100x100)

data = dict()
data["scaler"] = load(open("/Users/harry/pinns/models/tmp-scaler.pkl","rb"))

data = merge_features(data,geom_100x100)
data = predict_fluid(data,pinn,geom_100x100,args_100x100)

#################
##  CFD PINN   ##
#################

Setting up geometry...
Geometry summary:
	geom[x_start]: 0
	geom[x_end]: 1
	geom[numx]: 100
	geom[y_start]: 0
	geom[y_end]: 1
	geom[numy]: 100
	geom[t_start]: 0.0
	geom[t_end]: 5.0
	geom[t_dt]: 0.001
	geom[numt]: 5001
	geom[stride]: 1
	Geometry setup completed

Prediction of fluid properties...
	DEBUG: Applying scaler...
	DEBUG: Scaler applied
	DEBUG: len(x): 50010000
	DEBUG: len(y): 50010000
	DEBUG: len(t): 50010000
	Fluid properties prediction complete

